In [1]:
import pandas as pd
import os
import sys
import shutil
from thyroid.utils.exception import customException

In [2]:
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\aakas\Documents\Projects\Thyroid-disease-detection\research


In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\aakas\\Documents\\Projects\\Thyroid-disease-detection'

In [35]:
# define configuration
raw_directory = "raw_data"
artifact_directory = "artifact"
file_name = "allrep.data"
bad_directory = "bad_data"
ingested_directory = "ingested_data"


In [36]:
# Create the artifact directory if it doesn't exist
if not os.path.exists(artifact_directory):
    os.makedirs(artifact_directory)

# Create the ingested_data directory inside the artifact directory if it doesn't exist
ingested_data_directory = os.path.join(artifact_directory, ingested_directory)
if not os.path.exists(ingested_data_directory):
    os.makedirs(ingested_data_directory)


# Create the ingested_data directory inside the artifact directory if it doesn't exist
bad_data_directory = os.path.join(artifact_directory, bad_directory)
if not os.path.exists(bad_data_directory):
    os.makedirs(bad_data_directory)


In [37]:
import os

def read_data_file(file_path):
    if file_path.lower().endswith('.data' or '.txt'):
        try:
            with open(file_path, 'r') as file:
                data = file.readlines()
                print("read_data_file")
                return data
            
        except FileNotFoundError:
            return "File not found."
        except Exception as e:
            return "An error occurred: " + str(e)
    else:
        # if the file does not end with .data
        return False


In [38]:
import csv
from datetime import datetime

In [39]:
def save_to_csv(data, csv_path):
    try:
        print(len(data))
        with open(csv_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            #csv_writer.writerow(column_names)
            for row in data:
                csv_writer.writerow(row)
        return "CSV file saved successfully."
    except Exception as e:
        return "An error occurred while saving CSV: " + str(e)

In [40]:
import re
def process_rows(record):
    cleaned_attributes = []
    rows = record.strip().split('\n')  # Split input into rows
    for attribute in record.strip().split(','):
        attribute = re.sub(r'-', 'negative', attribute)  # Replace '-' with 'negative'
        if '.|' in attribute:
            cleaned_attributes.append(attribute.split('.|')[0])
        elif '[' in attribute:
            cleaned_attributes.append(attribute.split('[')[0])
        else:
            cleaned_attributes.append(attribute)
    return cleaned_attributes

In [41]:
# process the file extension
file_path = os.path.join(raw_directory, file_name)
data_rows = read_data_file(file_path)

read_data_file


In [42]:
data_rows

['41,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.3,t,2.5,t,125,t,1.14,t,109,f,?,SVHC,negative.|3733\n',
 '23,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,4.1,t,2,t,102,f,?,f,?,f,?,other,negative.|1442\n',
 '46,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.98,f,?,t,109,t,0.91,t,120,f,?,other,negative.|2965\n',
 '70,F,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.16,t,1.9,t,175,f,?,f,?,f,?,other,replacement therapy.|806\n',
 '70,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.72,t,1.2,t,61,t,0.87,t,70,f,?,SVI,negative.|2807\n',
 '18,F,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.03,f,?,t,183,t,1.3,t,141,f,?,other,negative.|3434\n',
 '59,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,?,f,?,t,72,t,0.92,t,78,f,?,other,negative.|1595\n',
 '80,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,2.2,t,0.6,t,80,t,0.7,t,115,f,?,SVI,negative.|1367\n',
 '66,F,f,f,f,f,f,f,f,f,f,f,f,t,f,f,t,0.6,t,2.2,t,123,t,0.93,t,132,f,?,SVI,negative.|1787\n',
 '68,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,2.4,t,1.6,t,83,t,0.89,t,93,f,?,SVI,negative.|2534\n',
 '84,F,f,f,f,f,f,f,f,f,f,f,f,t,f,f,t,1.1,t,2.2,t,115,t,0.95,t,121,f,?,SVI,

In [43]:
import yaml

In [44]:
from datetime import datetime

In [45]:
processed_data = []
if data_rows:
    print("Processing:", file_name)
    for row in data_rows:
        processed_data.append(process_rows(row))

        
    # convert file into .csv and save it to artifacts/ingested directory
    current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_file_name = f"thyroid_{current_datetime}.csv"
    csv_path = os.path.join(ingested_data_directory, csv_file_name)
    save_result = save_to_csv(processed_data, csv_path)
else:
    bad_file_path = os.path.join(bad_data_directory, file_name)
    shutil.move(file_path, bad_file_path)
    print("Moved to bad_data:", file_name)     

Processing: allrep.data
2800
